In [45]:
import tensorflow as tf
import numpy as np

def seq2seq_with_buckets(encoder_inputs, decoder_inputs, 
                         targets, target_weights, buckets, 
                         seq2seq_f, softmax_loss_function):
    return tf.contrib.legacy_seq2seq.model_with_buckets(encoder_inputs, decoder_inputs, 
                                                        targets, target_weights, buckets, 
                                                        seq2seq_f, softmax_loss_function = softmax_loss_function)

class Seq2SeqModel(object):
    
    def __init__(self,
                 vocab_size,
                 rnn_size,
                 num_layers,
                 buckets,
                 batch_size,
                 max_gradient_norm,
                 num_sampled, 
                 learning_rate,
                 lr_decay_factor,
                 forward_only):
        '''
        Args:
            与网络有关的超参数：
            vocab_size: 输入的维数
            rnn_size：隐层维数，它是embedding的维数，也是cell中状态的维数
            num_layers: rnn的层数
            与训练/图有关的超参数：
            buckets: 若干组不同的输入长度，将会建成若干组对应的图
            batch_size: 每个batch的大小
            max_gradient: 执行gradient clipping时的参数
            learning_rate, lr_decay_factor: 学习率与学习率衰减参数
            num_sampled: 计算loss时，不对整个vocab_size大小的output进行计算，而是采target中出现过的labels以及随机的num_samples个位置，计算loss
            forward_only: 是否只有前向没有后向
        '''
        
        # 赋值
        
        self.vocab_size = vocab_size
        self.rnn_size = rnn_size
        self.num_layers = num_layers
        self.buckets = buckets
        self.batch_size = batch_size
        self.max_gradient_norm = max_gradient_norm
        self.num_sampled = num_sampled
        
        # lr会改变，用一个variable和一个用于衰减的op lr_decay_op来控制
        
        self.learning_rate = tf.Variable(float(learning_rate), trainable = False)
        self.lr_decay_op = self.learning_rate.assign(self.learning_rate * lr_decay_factor)
        
        # global_step用于保存模型时的命名
        
        self.global_step = tf.Variable(0, trainable = False)
        
        # 在计算loss/预测时，需要把rnn_size的输出投影到vocab_size上，增加一个投影层
        
        proj_w = tf.get_variable("proj_w", [rnn_size, vocab_size])
        proj_b = tf.get_variable("proj_b", [vocab_size])
        
        w_t = tf.transpose(proj_w)
        
        output_proj = (proj_w, proj_b)
        
        # 用于计算之后的sampled_softmax_loss
        
        def sampled_loss(labels, logits):
            labels = tf.reshape(labels, [-1, 1])
            return tf.nn.sampled_softmax_loss(w_t, proj_b, labels = labels, inputs = logits, num_sampled = num_sampled, num_classes = vocab_size)
        
        # 构建网络里的单个cell
        
        cell =  tf.contrib.rnn.MultiRNNCell([tf.contrib.rnn.GRUCell(rnn_size) for _ in range(num_layers)])
        
        def seq2seq_f(encoder_inputs, decoder_inputs, do_decode):
            return tf.contrib.legacy_seq2seq.embedding_attention_seq2seq(
                encoder_inputs, 
                decoder_inputs,
                cell,
                num_encoder_symbols=vocab_size,
                num_decoder_symbols=vocab_size,
                embedding_size=rnn_size,
                output_projection=output_proj,
                feed_previous=do_decode)
        
        # 为encoder_inputs, decoder_inputs, target_weights建立placeholders
        
        self.encoder_inputs = []
        self.decoder_inputs = []
        self.target_weights = []
        self.targets = []
        
        # 这里的shape其实是后续的batch_size，如果不这样设置，去跑rnn的时候会出错QAQ
        for i in range(buckets[-1][0]):
            self.encoder_inputs.append(tf.placeholder(tf.int32, shape = [None], name = "encoder{0}".format(i)))
            
        # 由于target要由decoder_inputs移动一位得到，decoder_inputs需要多接收一个变量
        for i in range(buckets[-1][1] + 1):
            self.decoder_inputs.append(tf.placeholder(tf.int32, shape = [None], name = "decoder{0}".format(i)))
        for i in range(buckets[-1][1]):
            self.targets.append(self.decoder_inputs[i + 1])
            self.target_weights.append(tf.placeholder(tf.float32, shape = [None], name = "target{0}".format(i)))
            
        # 建立计算图，为每个bucket计算outputs, loss, 以及用于更新的op updates
        
       # self.outputs, self.losses = seq2seq_with_buckets(self.encoder_inputs, self.decoder_inputs, 
        #                                                 self.targets, self.target_weights, buckets, 
         #                                                lambda x, y: seq2seq_f(x, y, forward_only), sampled_loss)
        self.outputs, self.losses = tf.contrib.legacy_seq2seq.model_with_buckets(
          self.encoder_inputs, self.decoder_inputs, self.targets,
          self.target_weights, buckets, lambda x, y: seq2seq_f(x, y, True),
          softmax_loss_function=sampled_loss)
        
        if forward_only:
            # 此时所需要的是输出，于是将输出投影到vocab_size维上
            for i in range(len(buckets)):
                for j in range(len(self.outputs[i])):
                    self.outputs[i][j] = tf.matmal(self.outputs[i][j], proj_w) + proj_b
        else:
            # 计算每个bucket对应的更新——由于每个bucket对应的loss来源不同，这里相应地有不同的updates
            params = tf.trainable_variables()
            self.updates = []
            optimizer = tf.train.GradientDescentOptimizer(self.learning_rate)
            for i in range(len(buckets)):
                gradients = tf.gradients(self.losses[i], params)
                clipped_grad, _ = tf.clip_by_global_norm(gradients, max_gradient_norm)
                # 一次updates操作会让global_step增加1
                self.updates.append(optimizer.apply_gradients(zip(clipped_grad, params), self.global_step))
            
            # 保存所有变量
            self.saver = tf.train.Saver(tf.global_variables())

    def step(self, session, encoder_inputs, decoder_inputs, target_weights, bucket_idx, forward_only):
        '''训练/预测时，用encoder_inputs, decoder_inputs, target_weights跑一步。具体地，这里使用session.run([desired_outputs], feed_dict = {})来实现。
        Args:
            encoder_inputs, decoder_inputs, target_weights: 一组要喂的数据，都是二维的list，有序列长度个元素，每个元素是长度为batch_size的list
            bucket_idx: 这组数据对应bucket的下标
            forward_only: 若是，则用于预测，关心outputs，否则用于训练，关心updates
        Returns:
            outputs: 网络跑出的结果，是一个二维的list，有序列长度个元素，每个元素是长度为batch_size的list
            loss: 这个batch的loss
        '''
        enc_size, dec_size = self.buckets[bucket_idx]
        
        feed_dict = {}
        
        for i in range(enc_size):
            feed_dict[self.encoder_inputs[i].name] = encoder_inputs[i]
        for i in range(dec_size + 1):
            feed_dict[self.decoder_inputs[i].name] = decoder_inputs[i]
        for i in range(dec_size):
            feed_dict[self.target_weights[i].name] = target_weights[i]
            
        if forward_only:
            desired_output = []
            for i in range(dec_size):
                desired_output.append(self.outputs[bucket_idx][i])
            desired_output.append(self.losses[bucket_idx])
            out_all = sess.run(desired_output, feed_dict)
            # 返回outputs与loss
            return out_all[ : dec_size], out_all[dec_size] 
        else:
            desired_output = [self.updates[bucket_idx], self.losses[bucket_idx]]
            out_all = sess.run(desired_output, feed_dict)
            # 返回loss
            return out_all[1]

    

In [46]:
import data_utils as du
import random
import numpy as np
import tensorflow as tf

buckets = [(5, 5), (10, 10), (20, 20), (40, 40)]

vocab_size = 20000
batch_size = 8
rnn_size = 8
num_layers = 2
learning_rate = 0.5
lr_decay_factor = 0.99
max_gradient_norm = 5
num_samples = 256

def put_data_into_buckets(source, target):
    '''将source,target中的句对放入长度合适的bucket里
    Args:
        source, target: 两个list，每个元素是包含source/target中一个句子中词语的list
    Returns:
        encoder_inputs, decoder_inputs: 两个list，第i个元素是包含了长度落在第i个bucket中的句子的list
    '''

    encoder_inputs = [[] for _ in range(len(buckets))]
    decoder_inputs = [[] for _ in range(len(buckets))]

    drop_cnt = 0

    for line_s, line_t in zip(source, target):
        for i, (size_s, size_t) in enumerate(buckets):
            if len(line_s) < size_s and len(line_t) < size_t:
                encoder_inputs[i].append(line_s)
                decoder_inputs[i].append(line_t)
                break
        if len(line_s) >= buckets[-1][0] or len(line_t) >= buckets[-1][1]:
            '''
            line_s_words = ""
            for w in line_s:
                line_s_words += " " + vocab_list[w]
            line_t_words = ""
            for w in line_t:
                line_t_words += " " + vocab_list[w]
            print('Dropped:\n%s \n %s' % (line_s_words, line_t_words))
            '''
            drop_cnt += 1

    print('Sentence dropped: %d' % drop_cnt)
    for i in range(len(buckets)):
        print('Bucket %d: size = (%d, %d) cnt = %d' % (i, buckets[i][0], buckets[i][1], len(encoder_inputs[i])))

    return encoder_inputs, decoder_inputs

def get_batch(encoder_inputs_all, decoder_inputs_all, bucket_idx, batch_size):
    '''从encoder_inputs_all, decoder_inputs_all的第bucket_idx个bucket里，随机选出batch_size大小的句对
    Args:
        encoder_inputs_all, decoder_inputs_all: 两个个list，第i个元素是包含了所有长度落在第i个bucket中的句子的list
        bucket_idx: 要从中取句对的bucket的编号
        batch_size: batch的大小
    Returns:
        encoder_inputs, decoder_inputs: 两个分别包含了对应bucket_size个元素的list，第i个元素是一个长度为batch_size的list，代表每个句子第i个词
        target_weights: 用于决定计算loss时是否考虑结果的某个位：若target的该位是padding出来的，就置为0，否则置为1
        
    '''
    bucket_len = len(encoder_inputs_all[bucket_idx])
    enc_size, dec_size = buckets[bucket_idx]
    enc_lines = []
    dec_lines = []
    for _ in range(batch_size):
        idx = random.randint(0, bucket_len - 1)
        enc_line = encoder_inputs_all[bucket_idx][idx]
        dec_line = decoder_inputs_all[bucket_idx][idx]
        # encoder_input padding到对应长度即可
        enc_line = enc_line + [du.PAD_ID] * (enc_size - len(enc_line)) 
        # decoder_input padding到对应长度，再在前面加上<GO>，这样的长度比bucket长度多1，恰好可以产生bucket长度的target
        dec_line = [du.GO_ID] + dec_line + [du.PAD_ID] * (dec_size - len(dec_line)) 
        enc_lines.append(enc_line)
        dec_lines.append(dec_line)
    
    def list_transpose(inputs):
        '''将inputs代表的矩阵进行转置'''
        a = np.array(inputs)
        b = np.transpose(inputs)
        return list(list(c) for c in b)
    
    # 转置，使得能够用第0维索引找到所有句子在某处的词
    encoder_inputs = list_transpose(enc_lines)
    decoder_inputs = list_transpose(dec_lines)
    target_weights = []
    for len_idx in range(dec_size):
        target_weight = [1] * batch_size
        for batch_idx in range(batch_size):
            # 第batch_idx个句子的target的第len_idx位，对应了decoder_inputs的第len_idx + 1 位
            #  h   e  l  l  o  <EOS>  <-- target
            # []   [] [] [] []  []    <-- rnn cells
            # <GO> h  e  l  l   o     <-- decoder_inputs
            # 若这一位是<PAD>, 计算loss时它不计入，将weight置为0
            if (decoder_inputs[len_idx + 1][batch_idx] == du.PAD_ID):
                target_weight[batch_idx] = 0
        target_weights.append(target_weight)
    return encoder_inputs, decoder_inputs, target_weights

    


def build_model(session, forward_only):
    model = Seq2SeqModel(
                 vocab_size,
                 rnn_size,
                 num_layers,
                 buckets,
                 batch_size,
                 max_gradient_norm,
                 num_samples, 
                 learning_rate,
                 lr_decay_factor,
                 forward_only)
    ckpt = tf.train.get_checkpoint_state(FLAGS.train_dir)
    if ckpt and tf.train.checkpoint_exists(ckpt.model_checkpoint_path):
        print("Reading model parameters from %s" % ckpt.model_checkpoint_path)
        model.saver.restore(session, ckpt.model_checkpoint_path)
    else:
        print("Created model with fresh parameters.")
        session.run(tf.global_variables_initializer())
    return model

def train():
    filepaths = {
    'trn_src': './data/quora_duplicate_questions_trn.src',
    'trn_tgt': './data/quora_duplicate_questions_trn.tgt',
    'dev_src': './data/quora_duplicate_questions_dev.src',
    'dev_tgt': './data/quora_duplicate_questions_dev.tgt',
    'test_src': './data/quora_duplicate_questions_dev.src',
    'test_tgt': './data/quora_duplicate_questions_dev.tgt'
    }
    trn_src_ids, trn_tgt_ids, dev_src_ids, dev_tgt_ids, vocab_list = du.prepare_data(filepaths['trn_src'], filepaths['trn_tgt'], filepaths['dev_src'], filepaths['dev_tgt'])
    trn_encoder_inputs, trn_decoder_inputs = put_data_into_buckets(trn_src_ids, trn_tgt_ids)
    dev_encoder_inputs, dev_decoder_inputs = put_data_into_buckets(dev_src_ids, dev_tgt_ids)
    
    
    # 训练时，先随机一个bucket，再从其中随机一个batch，这里先计算每个bucket所含句子的个数，从而在之后使得每个bucket被随机到的概率与包含元素个数成比例
    trn_total = 0 # 训练集总大小
    trn_bucket_lens = [] # 每个bucket里训练集的大小列表
    for i in range(len(buckets)):
        trn_bucket_lens.append(len(trn_encoder_inputs[i]))
    trn_total = sum(trn_bucket_lens)
    
    tf.reset_default_graph()
    
    with tf.Session() as sess:
        model = build_model(sess, forward_only = False)
        
        current_time_step = 0
        
        while True:
            current_time_step += 1
            
            rand = random.randint(1, trn_total) 
            bucket_idx = -1
            for idx in range(len(buckets)):
                rand -= trn_bucket_lens[idx]
                if rand <= 0:
                    bucket_idx = idx
                    break
        
            encoder_inputs, decoder_inputs, target_weights = get_batch(trn_encoder_inputs, trn_decoder_inputs, bucket_idx, batch_size)
            cur_loss = model.step(sess, encoder_inputs, decoder_inputs, target_weights, bucket_idx, forward_only = False)


    '''
    encoder_inputs, decoder_inputs, target_weights = get_batch(trn_encoder_inputs, trn_decoder_inputs, 0, 15)

    print(encoder_inputs)
    print(decoder_inputs)
    print(target_weights)
    '''

train()

Original Vocabulary Size is 23904
Sentence dropped: 40
Bucket 0: size = (5, 5) cnt = 283
Bucket 1: size = (10, 10) cnt = 40895
Bucket 2: size = (20, 20) cnt = 70197
Bucket 3: size = (40, 40) cnt = 8004
Sentence dropped: 6
Bucket 0: size = (5, 5) cnt = 24
Bucket 1: size = (10, 10) cnt = 5098
Bucket 2: size = (20, 20) cnt = 8737
Bucket 3: size = (40, 40) cnt = 1029


TypeError: can't pickle _thread.lock objects